In [ ]:
from pyproj import CRS, Transformer

# Define the coordinate reference systems
osgb36 = CRS("EPSG:27700")  # OSGB36, the British National Grid
wgs84 = CRS("EPSG:4326")    # WGS84, used for GPS coordinates



In [ ]:
# Create a transformer to convert from OSGB36 to WGS84
transformer = Transformer.from_crs(osgb36, wgs84, always_xy=True)

# Example grid reference (Easting and Northing)
easting = 471750
northing = 118700

# Transform to latitude and longitude
beaconLong, beaconLat = transformer.transform(easting, northing)

print(f"Latitude: {beaconLat}, Longitude: {beaconLong}")


## GPX


In [ ]:
import pandas as pd
import gpxpy

# Read GPX file
with open('sdw.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract track points data
data = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            data.append([point.latitude, point.longitude, point.elevation, point.time])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Latitude', 'Longitude', 'Elevation', 'Time'])

# Show DataFrame
print(df)

### Folium visualisation

In [ ]:
import folium

first_point = gpx.tracks[0].segments[0].points[0]
m = folium.Map(location=[first_point.latitude, first_point.longitude], zoom_start=14)



In [ ]:
# Plot track
for track in gpx.tracks:
    for segment in track.segments:
        points = [(point.latitude, point.longitude) for point in segment.points]
        folium.PolyLine(points, color="red", weight=2.5, opacity=0.5).add_to(m)

# Display map
# m.save('GPX_track_map.html')
m  # if running in Jupyter, this will display the map inline

## The Fun Stuff
I want to find the total elevation gain between two points on the map. 

* Start of the hike
* Beacon Hill, first camp site:
\(50.9978318828897, -1.1406610794684595\). Stored in `beaconLat` and `beaconLong` variables defined in cell 2


In [ ]:
first_point

I want to find the index in the gpx that corresponds to Beacon Hill

In [ ]:
minDist = 100
closestPoint = None
beaconIdx = None
for idx, point in enumerate(gpx.tracks[0].segments[0].points):
    dist = ((point.latitude - beaconLat)**2 + (point.longitude - beaconLong)**2)**0.5
    if dist < minDist:
        minDist = dist
        closestPoint = point
        beaconIdx = idx

print(f"Closest point is {closestPoint} at index {beaconIdx} with distance {minDist}")

In [ ]:
# folium.Marker([closestPoint.latitude, closestPoint.longitude], popup='Beacon Hill').add_to(m)
# folium.Marker([beaconLat, beaconLong], popup='Measured Gridref').add_to(m)
coordinates = (-1.3185442160877858, 51.06329954208766), (-1.1406610794684595, 50.9978318828897), (-0.9796756545673202, 50.96326470728358), (-0.7706888049170811, 50.944885672656625), (-0.5403043488498133, 50.8969989412216)
colors = ['green', 'blue', 'blue', 'blue', 'red']
icons = ['star', 'map-marker', 'map-marker', 'map-marker', 'flag']
for p, color, icon in zip(coordinates, colors, icons):
    print(list(p[::-1]))
    folium.Marker(list(p[::-1]), icon=folium.Icon(color=color, icon=icon)).add_to(m)
m

In [ ]:
total_uphill = 0
route = gpx.tracks[0].segments[0].points
for i in range(beaconIdx):
    elevation_change = route[i+1].elevation - route[i].elevation
    if elevation_change > 0:
        total_uphill += elevation_change 

print(f"Total uphill: {total_uphill}m")